# Data science project in Python

Doormiddel van numpy, pandas, matplotlib & SciKit-learn

Door Louis D'Hont - Elektronica-ICT - 2021

## Project omschrijving

Het doel van deze opdracht is om op basis van bepaalde features het type klanten dat over gaat tot een nieuwe boeking te bepalen met een algoritme. Deze features zijn gehaald uit de hieronder beschreven data beschikbaar gesteld door TrailFinders. Ze organiseren paardrijvakanties over de hele wereld en helpen daarnaast ook om de vlucht, accomodatie, huurwagen, verzekeringen en dergelijke te regelen.

bepalen van de beste bestemming bepalen

In [1]:
#Importeren van gebruikte bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

#delete row that is not useful
#to_drop = ['Straat + nr.', 'Telefoon']
#df.drop(to_drop, inplace = True, axis = 1)()

#df_klanten['age'] = df_klanten['age'].astype('int64')


# Replace numerical value with string
#df['gender'].replace(0, 'Female', inplace=True)
#df['gender'].replace(1, 'Male', inplace=True)
# Replace all values that equal a specific variable
#df = df.replace(valueToReplace =["current", "alsoCurrent"],value ="newvalue")

## Data analyse

### Inlezen van data en features bepalen

Hieronder wordt de data die is aangeleverd behandeld.

In [2]:
klanten = pd.read_csv("../data/klanten.csv", sep=';') 
artikels = pd.read_csv("../data/artikels.csv", sep=';') 
bestellingen = pd.read_csv("../data/bestellingen.csv", sep=';') 
deelnemers = pd.read_csv("../data/deelnemers.csv", sep=';')

df_klanten = pd.DataFrame(klanten)
df_artikels = pd.DataFrame(artikels)
df_bestellingen = pd.DataFrame(bestellingen)
df_deelnemers = pd.DataFrame(deelnemers)


### Klantenlijst

Hieronder wordt de data uit de klantenlijst opgeruimd en aangepast.

In [3]:
df_klanten.dtypes

Familienaam        object
Voornaam           object
Straat + nr.       object
Telefoon           object
Klantnummer        object
Klantnummer CMS    object
Email              object
Creatie datum      object
Geb. datum         object
Locatie            object
Postcode           object
Land               object
dtype: object

In [4]:
#df_klanten['']

In [5]:
#check how many values are null
df_klanten.isnull().sum()

Familienaam           5
Voornaam             65
Straat + nr.         32
Telefoon           1334
Klantnummer           0
Klantnummer CMS    1656
Email                40
Creatie datum      1473
Geb. datum         2321
Locatie              31
Postcode             38
Land                  2
dtype: int64

After review, there is 1 volcano event that doesn't have an age. The record that has a null value for the year, also has a null value for location, country, lat, long, etc. This record does not help analysis and will be dropped.

To keep the default dataframe in tact, I've created a new dataframe and will clean up the data in this new dataframe. This is to ensure data is not skewed and can be compared after transformation.

### Data Quality
- Create new dataframe
 * Rename columns (change abbreviations to defined full words)
 * Change data types for columns
 * Handle null/NaN values


In [ ]:
# Create a new dataframe (deep copy) 
df = df_klanten.copy()
now = datetime.datetime.now()

def convert_datatypes(df):
    # Creatie datum
    # Geb. datum
    for index, date in enumerate(df['Geb. datum']):
        if (pd.notnull(date)):
            if (int(date[-4:]) > now.year):
                df.drop(df.index[index], inplace = True)
        #else:
            #print(str(date) + " " + str(index))
            #df.update(df_deelnemers)
   
            #for index, data in enumerate(df_deelnemers['Familienaam']): #Voornaam
                #df['Geb. datum'] = np.where(df['Familienaam'] == data, df_deelnemers['Geboortedatum'], 'False')
                #print(df['Geb. datum'])
    #df['Geb. datum'] = pd.to_datetime(df['Geb. datum'])
    #df['Geb. datum'] = df['Geb. datum'].astype('datetime64[ns]')
    
    
    
    
    
convert_datatypes(df)
#df.describe()
df.sample(20)

In [18]:

# De volgende is het samenvoegen van alle dataframes tot 1 goed dataframe

#1 - verwijder de kolommen die niet nuttig zijn
def remove_columns(df):
    df = df.drop(columns=['Straat + nr.', 'Locatie', 'Telefoon', 'Email', 'Voornaam'], inplace=True)
    
def convert_land(df):
    namen_belgie = ['belgium', 'belgië', 'belgïe', 'west-vlaanderen', 'b', 'belgie', 'belg', 'begie', 'ybelgie', 'be', 'oost-vlaanderen - belgium', 'oost-vlaanderen', 'belgiê', 'beglië']
    namen_nederland = ['nederand', 'nl', 'nederlands', 'ned', 'nerdland', 'holland', 'netherland', 'groningen', 'noord holland', 'noord-brabant', 'hengelo ov', 'nederland', 'nederlandse', 'the netherlands', 'pays bas (holland)', 'nerderland', 'nld', 'ndl', 'ned.', 'utrecht', 'noor holland', 'noord-holland', 'netherlands', 'zuid-holland', 'nederkand', 'nedeland']
    namen_frankrijk = ['france', 'frankrijk']
    namen_duitsland = ['duitsland', 'germany']
    namen_verenigd_koninkrijk = ['groot-brittanie ', 'united kingdom']
    namen_verwijderen = ['27', 'NaN', 'df', 'z', 'nan', 'Kies...', 'jelle', '- -', '-', 'test', '0633799726', 'Selecteer een staat', 'Selecteer een gebied, staat of provincie a.u.b.']

    #dit stuk moet korter
    df.drop(df.index[df['Land'] == 'NaN'], inplace = True)
    df.drop(df.index[df['Land'].isnull()], inplace = True)
    df.drop(df.index[df['Land'] == 'df'], inplace = True)
    df.drop(df.index[df['Land'] == '27'], inplace = True)
    df.drop(df.index[df['Land'] == 'z'], inplace = True)
    df.drop(df.index[df['Land'] == 'Kies...'], inplace = True)
    df.drop(df.index[df['Land'] == 'jelle'], inplace = True)
    df.drop(df.index[df['Land'] == '- -'], inplace = True)
    df.drop(df.index[df['Land'] == '-'], inplace = True)
    df.drop(df.index[df['Land'] == 'test'], inplace = True)
    df.drop(df.index[df['Land'] == '0633799726'], inplace = True)
    df.drop(df.index[df['Land'] == 'Selecteer een staat'], inplace = True)
    df.drop(df.index[df['Land'] == 'Selecteer een gebied, staat of provincie a.u.b.'], inplace = True)
    
    for index, land in enumerate(df['Land']):    
        #if str(land) in namen_verwijderen:
        #    print(index)
            
        if (str(land).lower()) in namen_belgie:
            df['Land'].iloc[index] = 'BE'
        if (str(land).lower()) in namen_nederland:
            df['Land'].iloc[index] = 'NL'
        if (str(land).lower()) in namen_frankrijk:
            df['Land'].iloc[index] = 'FR'
        if (str(land).lower()) in namen_duitsland:
            df['Land'].iloc[index] = 'DE'
        if (str(land).lower()) in namen_verenigd_koninkrijk:
            df['Land'].iloc[index] = 'GB'
        if (str(land).lower()) == 'luxemburg':
            df['Land'].iloc[index] = 'LUX'
        if (str(land).lower()) == 'scotland':
            df['Land'].iloc[index] = 'SCO'
        if (str(land).lower()) == 'portugal':
            df['Land'].iloc[index] = 'PT'
        if (str(land).lower()) == 'suriname':
            df['Land'].iloc[index] = 'SR'
        if (str(land).lower()) == 'hong kong':
            df['Land'].iloc[index] = 'CN'
        if (str(land).lower()) == 'oostenrijk':
            df['Land'].iloc[index] = 'AUT'
        if (str(land).lower()) == 'canada':
            df['Land'].iloc[index] = 'CA'
        if (str(land).lower()) == 'italy':
            df['Land'].iloc[index] = 'IT'
        if (str(land).lower()) == 'spanje':
            df['Land'].iloc[index] = 'ES'
        if (str(land).lower()) == 'chile':
            df['Land'].iloc[index] = 'CL'

remove_columns(df)
convert_land(df)

print("All unique values in the : " + str(df.Land.unique()))
print("Aantal rijen en kolommen: " + str(df.shape))
#df.describe()
df

All unique values in the : ['BE' 'NL' 'IT' 'FR' 'CA' 'DE' 'GB' 'CN' 'SCO' 'AUT' 'LUX' 'CL' 'SR' 'ES'
 'PT']
Aantal rijen en kolommen: (3659, 7)


,Familienaam,Klantnummer,Klantnummer CMS,Creatie datum,Geb. datum,Postcode,Land
0,8baed0e75af3232d,d963fdd785025ea9,NaN,11.02.2013,NaN,9860,BE
1,9777be93b58da2ca,ac983a70783ec68a,NaN,16.04.2020,NaN,1980,BE
2,9777be93b58da2ca,b43d39e962b43474,NaN,19.02.2013,NaN,1980,BE
3,c700680d4f35e3ea,a2c2445dfea02d1c,NaN,02.02.2014,NaN,2223,BE
4,fb6bf5a6ecbd3c14,2793b2960986ce2e,NaN,30.12.2011,NaN,5398CA,NL
...,...,...,...,...,...,...,...
3672,dfc9bda1b3a3d3f2,0c9726b68112728a,4c43d57f711f04b6,08.08.2017,NaN,1074BS,NL
3673,256705ae73575442,aa393e258363d27d,26d952ce32e65191,NaN,NaN,3972CG,NL
3674,97cb78f17ba8c677,b2c8419b8a9e1c57,0a9b14070a8a70c0,NaN,NaN,9690,NL
3675,1c259b1bd93c8aaa,56d28c901ad59627,c935133ffe024391,NaN,NaN,2651 sw,NL


In [203]:
df.sample(10)

# - verwijder Straat + nr. kolom
# Verander de kolom namen
# wijzig het type voor de 'Creatie datum' en 'Geb. datum' kolommen
# gebruik klantnummer als index voor het dataframe
# gebruik het Klantnummer voor de NaN waarden in Klantnummer CMS
# verwijder de initialen (bv: JL of BS) en spaties uit de Postcode kolom
# Zet de Land kolom om naar 1 land (Be, BE, Belgium,..) = België
# vervang NaN in Geboortedatum met de geboortedatum uit de deelnemers lijst waar de familienaam en voornaam overeenkomen


,Familienaam,Klantnummer,Klantnummer CMS,Creatie datum,Geb. datum,Locatie,Postcode,Land
586,ad9578a6b38a3ebd,3b9d21ae7ab3d292,3b0dc7f6e05de6e8,07.07.2018,NaN,Meerdonk,9170,BE
1166,3541b91650fc02ab,a024f61ec32e1458,26610e45e8686877,10.02.2016,NaN,Stevensweert,6107 BR,NL
106,7fc6367163b130f9,b701d64d7954c93a,NaN,23.04.2012,NaN,MORTSEL,2640,BE
1362,ff93401332086906,c2cc87a5ec84f9ff,NaN,19.09.2012,24.01.1979,Ijsselstein,3402 HW,NL
241,c9f6135e5fd2e6d4,42308a5ff2a3179d,280a1f017a216693,NaN,NaN,ZAANDAM,1506EZ,NL
2093,9ad16c2c9feb21a1,3514353a51d2d084,24b858a652c30d7c,NaN,NaN,Zeist,3703CL,NL
626,61b7dcae0b78d671,d33b0f6005716650,NaN,29.07.2013,NaN,KALMTHOUT,2920,BE
102,177acaf8722fc304,ad50f967a65bae29,2328cf75a7ca432d,NaN,NaN,Zuidwolde (Dr),7921 RR,NL
208,eabd07a39ee07ef4,2fe6a3b6013eb620,83e11d1393e6a30c,20.11.2017,24.03.1960,Hever,3191,BE
3151,80a5d7318c758358,b2b7f3361f87d558,32842bfd33430713,29.06.2017,NaN,Antwerpen,2018,BE


In [65]:
#df_klanten.shape

In [73]:
df_artikels.sample(10)

# verwijder ArticlePoNummer kolom
# zet Boekingsdatum en Vertrekdatum om naar juiste type datetime
# vervang Volledige naam klant door klantennummer uit klantenlijst door te zoeken met dossier nummer
# vervang DossierStatus 'Ok' met 1 of 0
# zet Artikelprijs en Totaal dossier om naar float
# maak dataframe (labels) met alle bestemmingen


,Dos. nr.,DossierStatus,Boekingsdatum,Vertrekdatum,Volledige naam klant,Bestemming,ArticlePoNummer,Artikelcode,Omschrijving,Artikelprijs,Totaal dossier
5301,798,Ok,22.02.2013 15:42:51,16.08.2013 00:00:00,aae803853ec93772,Moliets Frankrijk,NaN,9a54ba5423ffedcc,Poliskosten,"5,00","765,80"
7644,1884,Ok,17.05.2015 00:00:00,12.06.2015 00:00:00,098c046cb4c67620,Paardrijden op het strand in Zeeland,NaN,078cd33aaf1e5a44,"Weekend paardrijden 6 uur, logies en ontbijt","304,00","304,00"
584,2349,Ok,31.03.2016 00:00:00,17.04.2016 00:00:00,fdacb10b9cf9ceee,Paardrijden Costa de la Luz Spanje,NaN,f5c7887dc66050c7,Paardrijweek Costa de la Luz,"1.213,00","1.585,98"
10968,3424,Ok,16.01.2018 14:54:19,20.05.2018 00:00:00,b872dcaacce3a45c,Spanje,NaN,93c15333678b24c9,Trektocht in het land vd Conquistadores,"1.270,00","1.668,00"
3085,25,Ok,11.05.2011 13:56:58,02.09.2011 00:00:00,4d24b299b4e0ad61,Belgie Ardennen,NaN,79bb614e98f11d65,Paardrijweekend IJslanders Ardennen,"620,00","620,00"
11690,3854,Ok,05.07.2018 00:00:00,23.09.2018 00:00:00,705eb453d9a81028,Verenigde Staten,NaN,42917583a69cb7b1,Verzameltransfer retour USD 100 p.p. retour,"180,00","3.674,94"
5297,798,Ok,22.02.2013 15:42:51,16.08.2013 00:00:00,aae803853ec93772,Moliets Frankrijk,NaN,a91b026743fc98d0,Calamiteitenfonds,"2,50","765,80"
11240,3599,Ok,23.03.2018 14:12:58,28.04.2018 00:00:00,51f5a06a00e2e350,"Portugal, Azoren",NaN,e5bc97b89249e6aa,Groene Week Azoren,"2.830,00","2.716,73"
13442,20200317,Ok,08.10.2020 15:13:46,04.06.2021 00:00:00,38e71ba275d03185,Nederland,NaN,524a48e77f42796f,Paardrijden op Nederlands en Duits grondgebied,"0,00","355,00"
5008,679,Ok,27.11.2012 10:35:45,05.05.2013 00:00:00,aa7ff2720e52d244,Madrid Spanje,NaN,NaN,Lijnvlucht inclusief 20 kg bagage,"1.530,00","1.695,00"


In [12]:
#df_artikels.shape

In [60]:
#df_bestellingen.iloc[3308]
df_bestellingen.sample(20)

# Dos.jaar = 2021.0 - moet date worden en niet float zijn
# kolomnaam D.vertrek en Datum terug hernoemen
# status -1 en 0 omzetten naar 1 of 0
# verwijder laatste rij - is het totaal van alles opgeteld
# verwijder de regio kolom (veel NaN)
# verwijder de gefactureerd kolom of zet om naar 1 of 0 - type int64
# verwijder de Pakket Code kolom
# verwijder Status kolom
# verwijder Touroperator kolom (Waarschijnlijk niet relevant)
# Zet 'Datum creatie', 'D. vertrek' en 'Datum terug' om naar datetime type
# Zet bestemming en 'Land klant' om naar 1 land (Be, BE, Belgium,..) = België
# zet Totaalprijs en Openstaand dossier om naar het type int64
# verwijder # pass. kolom
# verwijder Openstaand dossier kolom
# verwijder Betalingen dossier kolom
# 

# status: 0 = bevestigd -1 = geannuleerd



,Status,Dos. jaar,Dos. nr.,Email,Klantnummer,Klantnummer CMS,Datum creatie,D. vertrek,Datum terug,Totaalprijs,Betalingen dossier,Openstaand dossier,Touroperator,Land klant,Bestemming,Regio,Reistype,# pass.,Pakket Code,Gefactureerd
2663,0.0,2017.0,3246,c1d7fd937902ad98,db828f76f9027aee,9f684406078356ab,04.09.2017 00:00:00,30.09.2017,07.10.2017,890,890,0,f5ecb98c8c5d97ac,nl,De Balkan trail,NaN,NaN,1.0,NaN,True
131,0.0,2011.0,168,6851669992a96ba9,9176c08d72c1f0bc,NaN,19.08.2011 15:37:21,27.08.2011,03.09.2011,1180,1180,0,f5ecb98c8c5d97ac,Nederland,Bulgarije,NaN,NaN,1.0,NaN,True
3907,0.0,2020.0,20200212,3301e21c4eb1f6b0,afcb339ed6632716,NaN,24.06.2020 22:10:10,10.07.2020,12.07.2020,510,510,0,3124a9d2236ff8eb,nl,Nederland,NaN,Ruitervakantie,2.0,NLMALW7P,True
3737,0.0,2019.0,20190572,58ff09752c6f72b3,0402aa14b1aa5ea1,NaN,26.12.2019 14:26:27,03.08.2020,07.08.2020,1150,1150,0,145a6c443d10d78c,nl,Frankrijk,NaN,Ruitervakantie,2.0,FRPAMSTPP,True
20,0.0,2011.0,31,68c0a4ff2d1cc8be,af54b0a5a5dd01cd,NaN,13.05.2011 15:47:39,12.07.2011,19.07.2011,"901,7","901,7",0,9178e7686e56d28f,Netherlands,Frankrijk Haut Languedoc,NaN,NaN,1.0,NaN,True
129,-1.0,2011.0,166,6b4bfa846590dddf,86f033218b01dc22,NaN,19.08.2011 12:22:57,11.09.2011,18.09.2011,"1901,85","1901,85",0,7526875b2e15e594,Nederland,Galway Ierland,NaN,NaN,1.0,NaN,True
2253,0.0,2017.0,2770,b6353804e27efd0a,f3e0a241f6cea4a1,2653bc934e2a44db,15.01.2017 00:00:00,19.08.2017,24.08.2017,730,730,0,4267f0cb3b5cc762,NETHERLANDS,Met de Huifkar door Ierland,NaN,NaN,5.0,NaN,True
2039,0.0,2016.0,2531,145f594d20195835,2a84452cf6f4362f,28040a840774dd65,27.06.2016 00:00:00,17.07.2016,22.07.2016,495,495,0,efbd72ddf0eb500d,Belgium,Paardrijden en Frans leren,NaN,NaN,1.0,NaN,True
3193,0.0,2018.0,3937,aa8641549110d09b,2b79001882723d33,2fcedf93f180a25c,03.09.2018 10:22:12,22.03.2019,24.03.2019,390,390,0,59fd5bafb3a477af,Nederland,"Belgie, Ardennen",NaN,Ruitervakantie,2.0,NaN,True
3312,0.0,2019.0,20190005,cfbf5d9446379df7,c61cd01c212424f6,d3f3e988a2f2261e,03.01.2019 09:56:57,28.04.2019,05.05.2019,2835,2835,0,38cd84b2be544d50,nl,Spanje,NaN,Ruitervakantie,3.0,ESANRAPTTP,True


In [28]:
#df_bestellingen.shape

In [45]:
df_deelnemers.sample(10)

# verwijder GSM kolom
# vewijder email kolom (veel NaN) - ook prive data
# zet Postcode om naar het type int64
# zet Vertrekdatum om naar datetime
# verwijder cijfers uit Stad bv: Brussel 15
# 

,Dossier,Vertrekdatum,Familienaam,Voornaam,Bestemming,Geboortedatum,Email,GSM,Straat + nr.,Postcode,Stad
1066,687,05.04.2013,844f363d7eb31148,730afa451c94a414,Brabant Leenderheide,25.08.1984,NaN,NaN,NaN,NaN,NaN
4848,3029,28.05.2017,deeb9e8e4be79b08,bc17e9a0fa641b62,"Spanje, andalusie",19.08.1986,NaN,NaN,NaN,NaN,NaN
3268,2082,09.04.2016,ecbfdfd4c23b50f3,61221a594232f995,Lissabon Portugal,NaN,NaN,NaN,NaN,NaN,NaN
5596,3567,26.05.2018,77ac08a9c066e88a,f0e40676f0cee5b7,Verenigde Staten van Amerika,22.01.1958,NaN,NaN,NaN,NaN,NaN
2468,1593,24.10.2014,04b50d4ea7ca208e,669c5147139dc3de,Paardrijden met IJslanders in de Ardennen,04.07.1969,NaN,NaN,NaN,NaN,NaN
7376,20200313,04.10.2020,3f535ba8d1a2a98c,b1711115c4a15933,Nederland,19.08.1967,NaN,NaN,NaN,NaN,NaN
6371,20190032,08.09.2019,eb67b7cad1fe686f,f8e9215bbc7296b9,Roemenie,06.05.1979,NaN,NaN,b8f625c05010b054,6871 EC,Renkum
2853,1811,19.07.2015,7580c4c256dbe896,646c285fa7f1d7ea,Paardrijden en Frans leren,07.05.2002,NaN,NaN,NaN,NaN,NaN
7257,20200202,31.07.2020,ff1b5d714ff1c0da,196c44711e214fc1,Frankrijk,18.10.2007,NaN,NaN,NaN,NaN,NaN
1990,1247,18.04.2014,829e4a69a43bc900,829e4a69a43bc900,Paardrijden met IJslanders in de Ardennen,16.01.1969,NaN,NaN,NaN,NaN,NaN


In [16]:
#df['Klantnummer'].is_unique

#make specific row as index
#df.set_index('Klantnummer', inplace=True)
#df

In [17]:
# aantal waarden die null zijn in de klantenlijst
#df.isnull().sum()

In [18]:
df.columns

NameError: name 'df' is not defined

In [19]:
#land lijst opruimen (België, belgium, BE) wijzigen naar 1 formaat
#achternaam van deelnemers kunnen worden gelinked met het klantennummer van een bestelling
#Geboortedatum in kantenlijst kan ge



In [20]:
df.dtypes

NameError: name 'df' is not defined

In [21]:
df.info()

NameError: name 'df' is not defined

In [72]:
#values count
#df.Email.value_counts(dropna=False)

In [71]:
df_bestellingen

,Status,Dos. jaar,Dos. nr.,Email,Klantnummer,Klantnummer CMS,Datum creatie,D. vertrek,Datum terug,Totaalprijs,Betalingen dossier,Openstaand dossier,Touroperator,Land klant,Bestemming,Regio,Reistype,# pass.,Pakket Code,Gefactureerd
0,-1.0,2021.0,0,0d484d11bfafa789,e7a25456cafdac84,NaN,04.01.2021 15:11:00,11.09.2021,18.09.2021,0,0,0,c723acdbab5e366e,nl,Spanje,NaN,NaN,1.0,ESOVFIHEXP,False
1,0.0,2011.0,1,36710cc356dbbc5f,756abea9c3e90efe,NaN,28.01.2011 15:01:35,10.07.2011,17.07.2011,"14,52","14,52",0,NaN,Belgium,Netherlands,NaN,NaN,1.0,NaN,True
2,0.0,2011.0,3,37459dca4934080f,cc44932a4b289e81,NaN,07.04.2011 10:17:02,13.05.2011,15.05.2011,620,620,0,dbaa85d88e7fdc52,Netherlands,Belgium,NaN,Autovakantie Europa,2.0,NaN,False
3,0.0,2011.0,4,fe82fc76bd814dcf,17a916e5222e39a0,NaN,08.04.2011 15:02:33,22.05.2011,28.05.2011,1620,1620,0,e11db4bae17e4d63,Nederland,Spain,NaN,NaN,0.0,NaN,False
4,0.0,2011.0,9,7418f4b177e773ac,94bc0881cd24a1b5,NaN,27.04.2011 15:01:28,08.05.2011,15.05.2011,5040,5040,0,NaN,Netherlands,Botswana,NaN,Safari,2.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,0.0,2021.0,20210039,a098cd64ef46d3e6,550888291b01a2d4,67e59fd90cae0e70,02.05.2021 16:15:33,16.06.2021,18.06.2021,335,0,335,3124a9d2236ff8eb,Nederland,Nederland,NaN,Weekendje weg,1.0,NLMALW9P,False
4035,-1.0,2021.0,20210040,9d9fcbd34601d5be,5ca1958fcc9b6d75,NaN,03.05.2021 12:42:36,09.05.2021,11.05.2021,440,0,440,27ef3947de4c44d2,nl,Nederland,NaN,NaN,2.0,NLSTWPAP,False
4036,0.0,2021.0,20210041,737f64db006800d3,19b236bfce3aba00,86359749d0859dbf,03.05.2021 14:12:47,11.06.2021,13.06.2021,315,0,315,5b0d4a8fc474f56d,Nederland,Nederland,NaN,Weekendje weg,1.0,NLRIJVPWP,False
4037,0.0,2021.0,20210043,b985f67e6b21ddb7,5436dead6768cd27,NaN,05.05.2021 10:33:07,25.07.2021,01.08.2021,6118,0,6118,0af23110f6efa1a3,Nederland,Kroatië,NaN,Ruitervakantie,3.0,NaN,False


In [27]:
#df.boxplot(column='Email', by='Telefoon', rot=90)

In [ ]:

    df['Land'] = np.where(land.str.lower() == 'belgium', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'belgië', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'belgïe', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'west-vlaanderen', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'b', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'belgie', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'begie', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'ybelgie', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'belg', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'be', 'BE', df['Land'])
    df['Land'] = np.where(land == 'OOST-VLAANDEREN - BELGIUM', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'oost-vlaanderen', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'belgiê', 'BE', df['Land'])
    df['Land'] = np.where(land.str.lower() == 'beglië', 'BE', df['Land'])

# Features bepalen

Enkele interessante features die in de data te vinden zijn:

* Feature1

### importeren

Voordat 